# Massey Ratings
    - Matt Robinson, Yale Undergraduate Sports Analytics Group

This notebook is one of several notebooks exploring ratings systems often used in sports. All of the notebooks can be found in [this repo](https://github.com/mc-robinson/Ratings_Methods). 

Specifically, this notebook attempts to both explain and implement the popular Massey ratings model created by [Ken Massey](http://www.masseyratings.com/). Legend has it that Massey, now a math professor, came up with this method while still an undergraduate (which I include as a fun fact and as motivation for me to figure out my life).

## The Basic Overview ##

Massey's method is also referred to as the *Point Spread Method*. To put it simply, the goal of this method is for the difference in ratings between two teams to equal the difference in score when they play each other. Mathematically, this can be easily summarized by the equation:
$$ r_i - r_j = y_k$$
where $r_i$ and $r_j$ are the ratings of teams $i$ and $j$, respectively, and $y_k$ is the point differential for game $k$, in which $i$ and $j$ play each other.

Now, of course, this equation is just the ideal. Over the course of a few games, you will get a system of equations that look just like it. Finding ratings such that it holds true for every team and every game is unrealistic. Therefore, Massey's method uses what is called the *method of least squares* to find the best approximate ratings.

## Working Through an Example ##

In order to work through the implementation of this method and understand how it works, I'm going to use an extremely simplified example. Let's imagine the Ivy league is instead the IV league and consists of only four teams who each play each other once: Harvard, Princeton, Yale, and Columbia.

Here are the results from the 2016 IV season that I scraped from the NCAA stats website:


In [42]:
import numpy as np
import pandas as pd

In [43]:
IV_df = pd.read_csv('IV_league_2016.csv')

In [44]:
IV_df

,year,month,day,team,opponent,location,team_score,opponent_score
0,2016,10,1,Columbia,Princeton,1,13,48
1,2016,10,22,Harvard,Princeton,-1,23,20
2,2016,10,28,Columbia,Yale,1,23,31
3,2016,11,5,Columbia,Harvard,-1,21,28
4,2016,11,12,Princeton,Yale,-1,31,3
5,2016,11,19,Harvard,Yale,1,14,21


The `location` column tells us if the team in the `team` column was home or away. 
* Away = -1
* Neutral = 0
* Home = 1

Since the goal of Massey's method is to find ratings whose difference gives the score differential, we need to add a column with this data:

In [45]:
IV_df['score_diff'] = (IV_df['team_score']-IV_df['opponent_score'])

In [46]:
IV_df

,year,month,day,team,opponent,location,team_score,opponent_score,score_diff
0,2016,10,1,Columbia,Princeton,1,13,48,-35
1,2016,10,22,Harvard,Princeton,-1,23,20,3
2,2016,10,28,Columbia,Yale,1,23,31,-8
3,2016,11,5,Columbia,Harvard,-1,21,28,-7
4,2016,11,12,Princeton,Yale,-1,31,3,28
5,2016,11,19,Harvard,Yale,1,14,21,-7


### The Setup ###

Now comes time to get back to our original equation:
$$ r_i - r_j = y_k $$

In this case, we are going to try to find the ratings for all four teams of the IV league: $r_h, r_p, r_y,$ and $r_c$. To do this, we look at the six league games from the IV season and construct an equation for each game. In our case, the system of equations looks like this (using the data from the table above):

$$ 
\begin{align*}
r_c - r_p &= -35 \\
r_h - r_p &=  3 \\
r_c - r_y &= -8 \\
r_c - r_h &= -7 \\
r_p - r_y &= 28 \\
r_h - r_y &= -7 \\
\end{align*}
$$


For sake of simplicity, I am going to take every equation with a negative `score_diff` and multiply through by $-1$. Now every point differential is positive and the system of equations looks like this:
$$ 
\begin{align*}
r_p - r_c &= 35 \\
r_h - r_p &=  3 \\
r_y - r_c &= 8 \\
r_h - r_c &= 7 \\
r_p - r_y &= 28 \\
r_y - r_h &= 7 \\
\end{align*}
$$

Let's now encode this system of equations in matrix form. Generally, in a season of $m$ games among $n$ different teams, there will be $m$ equations (one for each game) and $n$ unknowns (the ratings). Thus, we can create an $m \times n$ matrix to encode the equations from the $m$ games and $n$ teams. For our example with six games and four teams, the matrix $X$ looks like this:

In [47]:
X = np.array([
        [0, 1, 0, -1],
        [1, -1, 0, 0],
        [0, 0, 1, -1],
        [1, 0, 0, -1],
        [0, 1, -1, 0],
        [-1, 0, 1, 0]
    ])
X

array([[ 0,  1,  0, -1],
       [ 1, -1,  0,  0],
       [ 0,  0,  1, -1],
       [ 1,  0,  0, -1],
       [ 0,  1, -1,  0],
       [-1,  0,  1,  0]])

where each of the columns corresponds to a team, and each row is a game. The labeled output may be more clear:

In [48]:
game_num = ['game_' + _ for _ in '123456']
team_names = ['Harvard', 'Princeton', 'Yale', 'Columbia']
X_df = pd.DataFrame(X, index=game_num, columns=team_names)
print(X_df.to_string())

        Harvard  Princeton  Yale  Columbia
game_1        0          1     0        -1
game_2        1         -1     0         0
game_3        0          0     1        -1
game_4        1          0     0        -1
game_5        0          1    -1         0
game_6       -1          0     1         0


For example, the first row corresponds to the game between Princeton and Columbia. The winning team, Princeton, gets a $1$ in its column while the losing team, Columbia, gets a $-1$ in its column.

Our full system of equations can be written using the linear matrix equation $Xr=y$, where $X$ is the $m \times n$ matrix we just constructed, $r$ is the $n \times 1$ vector containing the ratings of each team, and $y$ is the $m \times 1$ vector of score differences from each game.

$$
\begin{bmatrix}
  0 & 1 & 0 & -1 \\
  1 & -1 & 0 & 0 \\
  0 & 0 & 1 & -1 \\
  1 & 0 & 0 & -1 \\
  0 & 1 & -1 & 0 \\
  -1 & 0 & 1 & 0
\end{bmatrix} 
\begin{bmatrix}
  r_h \\
  r_p \\
  r_y \\
  r_c
\end{bmatrix}
= 
\begin{bmatrix}
  35 \\
  3 \\
  8 \\
  7 \\
  28 \\
  7
\end{bmatrix}
$$

Now, you can see that it is pretty easy to construct this matrix equation from the beginning. For each game (row), just put a $1$ in the column of the winning team, a $-1$ in the column of the losing team, and encode the absolute value of the point differential (the margin of victory) in the $y$ vector.

### Solving for the Ratings ###

In any given season, it is likely that the number of games played ($m$) will be much greater than the number of teams ($n$). This means that we have more equations than unknowns. In our IV league example, we had six equations but only four unknowns. Mathematically, this means that the matrix is *overdetermined*, and thus the system of equations is likely *inconsistent*. In simpler words, this means that there is almost certainly no solution to the system of equations.

#### Unnecsary Aside: A simple example for those unfamiliar with least squares ####
Take this really simple example of two equations and one unknown. 
$$
\begin{align*}
x &= 5 \\
x &= 9
\end{align*}
$$

Obviously, there is no $x$ that satisfies both of these equations. So we are going to need to look for the best approximate solution $\bar x$. But how do we measure which is the "best" solution?

Often, we consider the best fit to be the one that minimizes the *sum of the squared errors*. To be more mathematical, for our system of $m=2$ and $n=1$ unknowns, we seek the $\hat x$ that minimizes the following:
$$
\sum_{i=1}^m (x-\bar x)^2 = (9-\bar x)^2 + (5- \bar x)^2
$$
Now I think it is intuitive that the solution is somewhere between $5$ and $9$. The first guess is probably the average of $5$ and $9$, $\bar x = 7$ But some of you may think that it would be more advantageous to choose $\bar x = 6$ or  $\bar x = 8$ since they are closer to one of the numbers, or perhaps even $\bar x = 5$ or  $\bar x = 9$ since it makes one of the eqations exactly true. Well, it turns out that the average $\bar x = 7$ is the best choice because it minimizes $\sum_{i=1}^m (x-\bar x)^2$ if you do the math out. (This is no coincidence; the *mean* is precisely that estimator that minimizes the sum of squared errors)  

So this is exactly what we are trying to with our matrix equation: find those ratings that give the best approximate answer to our system of equations, as determined by the least squares criterion.

#### The least squares solution to our matrix equation #### 

So in the case of our matrix equation $Xr=y$ that we cannot find a solution for, we seek the least squares solution $\hat r$ such that the error $E = \parallel{Xr - y}\parallel^2$ is minimized.  

To find this $\hat r$ we focus on solving what are called the "normal equations," $X^TX\hat r = X^T y$. 

#### Unnecessary Aside: Deriving the Normal Equations ####

In the machine learning courses I have seen, the normal equations are often derived using calculus. However, I find matrix calculus a bit narly, so instead I am going to use some geometry to derive them. If you want to see the calculus derivation, please see a linear algebra/machine learning textbook.

When we are trying to solve $Xr=y$, a solution exists if and only if a linear combination of the columns of $X$ can equal the $y$ vector. That is to say (in slightly more mathematical jargon), a solution exists if $y$ lies in the column space of $X$, which I'll call $C(X)$. Well since we have an overdetermined system, $y$ is almost definitely outside of $C(X)$. So what do we do? The answer is that we give up on solving exactly for $y$. Instead, let's look for $X\hat r$, the vector in $C(X)$ that is as close as possible to $y$. 

But what does close mean? It means that the error vector $e = (y - X \hat r)$ is perpendicular to the vector $X \hat r$, which is in $C(X)$. (If the reason for this is not clear to you, this [medium post](https://medium.com/@andrew.chamberlain/the-linear-algebra-view-of-least-squares-regression-f67044b7f39b) has a rather good illustration)

And when two vectors are perpendicular, we know that their dot product must be $0$. So let's do it out and solve for the approximate ratings vector $\hat r$:

$$
\begin{align*}
(X \hat r)^T (y - X \hat r) &= 0 \\
\hat r^T X^T y -\hat r^T X^T X \hat r &= 0 \\
\hat r^T X^T X \hat r &= \hat r^T X^T y \\
(\hat r^T)^{-1} \hat r^T X^T X \hat r &= (\hat r^T)^{-1} \hat r^T X^T y \\
X^T X \hat r &= X^T y \\ 
\end{align*}
$$
And, if $X^T X$ is invertible:
$$
\hat r = (X^T X)^{-1} X^T y 
$$

If anyone cares, the matrix $(X^T X)^{-1} X^T$ is often referred to as the *Moore-Penrose Pseudoinverse*.

#### Back to the example

Anyways, let's do the math out for our IV league example and attempt to solve the normal equations $X^TX\hat r = X^T y$:

First examine what the matrix product $X^TX$ looks like:

In [49]:
print("X looks like:")
print(X_df.to_string() + '\n')
print("X^T looks like:")
print(X_df.T.to_string())

X looks like:
        Harvard  Princeton  Yale  Columbia
game_1        0          1     0        -1
game_2        1         -1     0         0
game_3        0          0     1        -1
game_4        1          0     0        -1
game_5        0          1    -1         0
game_6       -1          0     1         0

X^T looks like:
           game_1  game_2  game_3  game_4  game_5  game_6
Harvard         0       1       0       1       0      -1
Princeton       1      -1       0       0       1       0
Yale            0       0       1       0      -1       1
Columbia       -1       0      -1      -1       0       0


Let's call the matrix product $M$ for short

In [50]:
M = (X.T).dot(X)
M

array([[ 3, -1, -1, -1],
       [-1,  3, -1, -1],
       [-1, -1,  3, -1],
       [-1, -1, -1,  3]])

Showing the labels that each row and column correspond to:

In [51]:
team_names = ['Harvard', 'Princeton', 'Yale', 'Columbia']
M_df = pd.DataFrame(M, index=team_names, columns=team_names)
print(M_df.to_string())

           Harvard  Princeton  Yale  Columbia
Harvard          3         -1    -1        -1
Princeton       -1          3    -1        -1
Yale            -1         -1     3        -1
Columbia        -1         -1    -1         3


It may also help to see the whole multiplication written out:

$$
M = X^TX =
\begin{bmatrix}
  0 & 1 & 0 & 1 & 0 & -1 \\
  1 & -1 & 0 & 0 & 1 & 0 \\
  0 & 0 & 1 & 0 & -1 & 1 \\
  -1 & 0 & -1 & -1 & 0 & 0
\end{bmatrix} 
\begin{bmatrix}
  0 & 1 & 0 & -1 \\
  1 & -1 & 0 & 0 \\
  0 & 0 & 1 & -1 \\
  1 & 0 & 0 & -1 \\
  0 & 1 & -1 & 0 \\
  -1 & 0 & 1 & 0
\end{bmatrix} 
=
\begin{bmatrix}
  3 & -1 & -1 & -1 \\
  -1 & 3 & -1 & -1 \\
  -1 & -1 & 3 & -1 \\
  -1 & -1 & -1 & 3
\end{bmatrix}  
$$

$M$ is an $n \times n$ matrix. In general, the diagonal entries $M_{ii}$ correspond to the total number of games played by team $i$. The other entries $M_{ij}$ for $i \neq j$ are simply equal to the negation of how many times team $i$ played team $j$.

Now let's move on to the right side of the equation, $X^T y$. For sale of simplicity, we will call this vector $p$.

In [52]:
yT = np.array([
        [35, 3, 8, 7, 28, 7]
    ])
y = yT.T
y

array([[35],
       [ 3],
       [ 8],
       [ 7],
       [28],
       [ 7]])

In [53]:
p = (X.T).dot(y)
p

array([[  3],
       [ 60],
       [-13],
       [-50]])

In [54]:
team_names = ['Harvard', 'Princeton', 'Yale', 'Columbia']
p_df = pd.DataFrame(p, index=team_names,columns=[''])
print(p_df.to_string())

             
Harvard     3
Princeton  60
Yale      -13
Columbia  -50


Some of you may notice something special about this vector $p$. I'll write the multiplication out in full so that it may be more clear to you.

$$
p = X^Ty =
\begin{bmatrix}
  0 & 1 & 0 & 1 & 0 & -1 \\
  1 & -1 & 0 & 0 & 1 & 0 \\
  0 & 0 & 1 & 0 & -1 & 1 \\
  -1 & 0 & -1 & -1 & 0 & 0
\end{bmatrix} 
\begin{bmatrix}
  35 \\
  3 \\
  8 \\
  7 \\
  28 \\
  7
\end{bmatrix}
=
\begin{bmatrix}
  3 \\
  60 \\
  -13 \\
  -50 
\end{bmatrix} 
$$

The vector $p$ encodes the total point differential, summed across every game for a given team. Let's just examine the first row of $p$, which encodes Harvard's total point differential. 

Harvard played three games in the IV league. In their first game they beat Princeton with a point differential of +3. In their second game, they beat Columbia with a point differential of +7. In their last game, they lost to Yale by a point differential of -7. Alltogether, their total point is 7 + 3 -7 = 3.

If you do the matrix multiplaction for the first row, the same exact thing is happening. You add all the point differentials from Harvard's win and subtract the point differentials from the loss. Thus, $p$ simply encodes the cumulative point differentials for each team.

Now, it is also pretty easy to see that both $M$ and $p$ can be constructed pretty simply from the beginning without having to first construct $X$ and $y$. This avoids some unnecessary computation. Also note that $M$ is $n \times n$ and is thus ususally much smaller than the $m \times n$ matrix $X$.

#### Finally Solving the Normal Equations.

Now that we have seen how to construct $M$ and $p$, let's solve for $\hat r$ using the normal equations $M \hat r = p$.

$$
\begin{bmatrix}
  3 & -1 & -1 & -1 \\
  -1 & 3 & -1 & -1 \\
  -1 & -1 & 3 & -1 \\
  -1 & -1 & -1 & 3
\end{bmatrix}
\begin{bmatrix}
  r_h \\
  r_p \\
  r_y \\
  r_c
\end{bmatrix}
=
\begin{bmatrix}
  3 \\
  60 \\
  -13 \\
  -50 
\end{bmatrix} 
$$

(Note: I probably should put hats over the individual ratings in $\hat r$, but by now I think it is clear that these ratings will be an approximation.)

But before we solve for the ratings, it is worth also examining the columns of $M$ to spot **one minor issue**. Some of you may have seen immedietly this interesting property of the columns of $M$:

$$
\begin{bmatrix}
   3 \\
  -1 \\
  -1 \\
  -1 
\end{bmatrix}  
+ 
\begin{bmatrix}
  -1 \\
  3 \\
  -1 \\
  -1 
\end{bmatrix} 
+
\begin{bmatrix}
  -1 \\
  -1 \\
  3 \\
  -1 
\end{bmatrix} 
+ 
\begin{bmatrix}
  -1 \\
  -1 \\
  -1 \\
  3 
\end{bmatrix} 
= 
\begin{bmatrix}
  0 \\
  0 \\
  0 \\
  0 
\end{bmatrix} 
$$

Some of you will also remember that a set of vectors is *linearly dependent* if one of the vectors can be written as a linear combination of the others vectors. Well let me just move one of the vectors over to the other side and multiply the equation by $-1$:

$$
-1*
\begin{bmatrix}
   3 \\
  -1 \\
  -1 \\
  -1 
\end{bmatrix}  
+ 
-1*
\begin{bmatrix}
  -1 \\
  3 \\
  -1 \\
  -1 
\end{bmatrix} 
+
-1*
\begin{bmatrix}
  -1 \\
  -1 \\
  3 \\
  -1 
\end{bmatrix} 
= 
\begin{bmatrix}
  -1 \\
  -1 \\
  -1 \\
  3 
\end{bmatrix} 
$$

It's now pretty obvious that this set of vectors is linearly dependent. You may similarly notice that the same is true of the rows of $M$. These properties also hold true in general when we consider the matrix $M$ over an arbitrary set of games and teams.

The linear dependence of the columns of $M$ has some interesting properties that we need to consider. The first thing you may remember from a linear algebra class is that linearly dependent columns is one of the many, many equivalent ways to say that a matrix is not invertible. Also, we observe that $rank(M) < n$, and thus the solution for $\hat r$ is not unique.

Massey came up with a **clever fix** for this. He simply replaces the last row of $M$ with a row of all ones. and sets the last entry of $p$ to be $0$. This new system is now denoted $\bar M \hat r = \bar p$ and looks like this:

$$
\begin{bmatrix}
  3 & -1 & -1 & -1 \\
  -1 & 3 & -1 & -1 \\
  -1 & -1 & 3 & -1 \\
  1 & 1 & 1 & 1
\end{bmatrix}
\begin{bmatrix}
  r_h \\
  r_p \\
  r_y \\
  r_c
\end{bmatrix}
=
\begin{bmatrix}
  3 \\
  60 \\
  -13 \\
  0 
\end{bmatrix} 
$$

$\bar M$ is now full rank and there is a unique solution for $\hat r$. There is one other nice property of this new setup which can be seen by doing the matrix multiplication for the last row. 
$$
1*r_h + 1*r_p + 1*r_y + 1*r_c = 0
$$
That is, all of the rankings must now sum to zero.

Let's now finally get a solution using numpy for the ratings:

In [58]:
M_bar = M.copy()
M_bar[-1,:] = np.ones(M.shape[0])
M_bar

array([[ 3, -1, -1, -1],
       [-1,  3, -1, -1],
       [-1, -1,  3, -1],
       [ 1,  1,  1,  1]])

In [59]:
p_bar = p
p_bar[-1] = 0
p_bar

array([[  3],
       [ 60],
       [-13],
       [  0]])

In [60]:
r_hat = np.linalg.inv(M_bar).dot(p_bar)
r_hat

array([[  0.75],
       [ 15.  ],
       [ -3.25],
       [-12.5 ]])

In [61]:
team_names = ['Harvard', 'Princeton', 'Yale', 'Columbia']
r_df = pd.DataFrame(r_hat, index=team_names,columns=[''])
print(r_df.to_string())

                
Harvard     0.75
Princeton  15.00
Yale       -3.25
Columbia  -12.50


## Some Additional Features

### Offense and Defense Vectors

One of the coolest features of Massey's method is the ability to split the overall ratings vector $r$ into an offensive ratings vector $o$ and a defensive ratings vector $d$. Massey's simple assumption is that the each team rating $r_i$ can be considered the sum of its offensive and defensive ratings:
$$
r_i = o_i + d_i
$$

In our example, the vector $r=o+d$ looks as follows:

$$
\begin{bmatrix}
  r_h \\
  r_p \\
  r_y \\
  r_c
\end{bmatrix}
=
\begin{bmatrix}
  o_h \\
  o_p \\
  o_y \\
  o_c
\end{bmatrix}
+
\begin{bmatrix}
  d_h \\
  d_p \\
  d_y \\
  d_c
\end{bmatrix}
$$

To solve for these offensive and defensive ratings, we also need to do a few things to the right hand side of the equation $Mr = p$. As we know from before, $p$ is the vector of cumulative point differentials $p_i$ for each team $i$. We can decompose each $p_i$ into the total points scored by team $i$ minus the total points scored against $i$. In vector form, we set $p=f-a$, where $p$ is the 'points for" vector, and $a$ is the points against vector. 

In our IV example, $p=f-a$ is decomposed as follows:

$$
\begin{bmatrix}
  3 \\
  60 \\
  -13 \\
  -50 
\end{bmatrix} 
= 
\begin{bmatrix}
  65 \\
  99 \\
  55 \\
  57 
\end{bmatrix} 
-
\begin{bmatrix}
  62 \\
  39 \\
  68 \\
  107 
\end{bmatrix} 
$$




Before we can continue, we also need to decompose $M$, which we call the "Massey coefficient matrix." We split $M$ into $M=T-P$, where $T$ is a diagonal matrix and $P$ is an off-diagonal matrix. $T$ holds the total number of games played by each team on the diagonal entries. $P$ meanwhile, holds the number of times two teams play each other in a season.

$M=T-P$ for our IV example looks as follows:

$$
\begin{bmatrix}
  3 & -1 & -1 & -1 \\
  -1 & 3 & -1 & -1 \\
  -1 & -1 & 3 & -1 \\
  -1 & -1 & -1 & 3
\end{bmatrix}
= 
\begin{bmatrix}
  3 & 0 & 0 & 0 \\
  0 & 3 & 0 & 0 \\
  0 & 0 & 3 & 0 \\
  0 & 0 & 0 & 3
\end{bmatrix}
-
\begin{bmatrix}
  0 & 1 & 1 & 1 \\
  1 & 0 & 1 & 1 \\
  1 & 1 & 0 & 1 \\
  1 & 1 & 1 & 0
\end{bmatrix}
$$

We now work with the original system $Mr=p$ to derive the $o$ and $d$ vectors.

$$
\begin{align*}
Mr &= p \\
(T-P)r &= p \\
(T-P)(o+d) &= p \\
To-Po+Td-Pd &= p \\
To-Po+Td-Pd &= f-a
\end{align*}
$$

We split the final equation into two separate equations:

$To-Pd=f$ and $Po-Td=a$

Let's look at these equations in turn:

First, the left equation $To-Pd=f$:

$$
\begin{bmatrix}
  3 & 0 & 0 & 0 \\
  0 & 3 & 0 & 0 \\
  0 & 0 & 3 & 0 \\
  0 & 0 & 0 & 3
\end{bmatrix}
\begin{bmatrix}
  o_h \\
  o_p \\
  o_y \\
  o_c
\end{bmatrix}
-
\begin{bmatrix}
  0 & 1 & 1 & 1 \\
  1 & 0 & 1 & 1 \\
  1 & 1 & 0 & 1 \\
  1 & 1 & 1 & 0
\end{bmatrix}
\begin{bmatrix}
  d_h \\
  d_p \\
  d_y \\
  d_c
\end{bmatrix}
=
\begin{bmatrix}
  65 \\
  99 \\
  55 \\
  57 
\end{bmatrix} 
$$

Let's just do the multiplication out:

$$
3o_h - d_p - d_y - d_c = 65 \\
3o_p - d_h - d_y - d_c = 99 \\
3o_y - d_h - d_p - d_c = 55 \\
3o_c - d_h - d_p - d_y = 57
$$

All these equations say is that the total number of points scored by one team in a season should equal the number of games played times their offensive rating minus the defensive ratings for each of the teams they played. For example, the first row says that the 65 points Harvard scored throughout the IV season should equal 3 times their offensive ratings minus the sum of the defensive ratings of Princeton, Yale, and Columbia (their opponents in the 3 games).

Let's work with this equation some more:

$$
\begin{align*}
To-Pd &= f \\
T(r-d)-Pd &= f \\
(T+P)d &= Tr-f
\end{align*}
$$

The right hand side of this equation, $Tr-f$, can be computed quite easily since we already know $T$,$r$, and $f$. Thus we just have to solve this system to find $d$, then we can find $o$ from $o$ = $r$ - $d$.

Let's solve it for our IV example:

In [62]:
T = np.diag(np.diag(M))
T

array([[3, 0, 0, 0],
       [0, 3, 0, 0],
       [0, 0, 3, 0],
       [0, 0, 0, 3]])

In [63]:
P = T.copy() - M.copy()
P

array([[0, 1, 1, 1],
       [1, 0, 1, 1],
       [1, 1, 0, 1],
       [1, 1, 1, 0]])

In [68]:
fT = np.array([
        [65, 99, 55, 57]
    ])
f = fT.T
f

array([[65],
       [99],
       [55],
       [57]])

In [70]:
d = np.linalg.inv(T+P).dot(T.dot(r_hat)-f)
d

array([[ -8.375],
       [ -4.   ],
       [ -9.375],
       [-24.25 ]])

In [72]:
team_names = ['Harvard', 'Princeton', 'Yale', 'Columbia']
d_df = pd.DataFrame(d, index=team_names,columns=[''])
print("defensive ratings d_i for each team:")
print(d_df.to_string())

defensive ratings d_i for each team:
                 
Harvard    -8.375
Princeton  -4.000
Yale       -9.375
Columbia  -24.250


In [74]:
o = r_hat.copy() - d.copy()
o

array([[  9.125],
       [ 19.   ],
       [  6.125],
       [ 11.75 ]])

In [75]:
team_names = ['Harvard', 'Princeton', 'Yale', 'Columbia']
o_df = pd.DataFrame(o, index=team_names,columns=[''])
print("offensive ratings o_i for each team:")
print(o_df.to_string())

offensive ratings o_i for each team:
                 
Harvard     9.125
Princeton  19.000
Yale        6.125
Columbia   11.750


Massey considers these offensive ratings $o_i$ to be "the number of points team $i$ would score against an average defense." Likewise, the defensive rating ?????$d_i$ for each team $i$ can be thought of as the number of points team $i$ would give up against an average offense. ??????

Another nice thing about the vecotrs $o$ and $d$ is that we can use them to predict the point outcomes for each game. For example, let's take a theoretical matchup of Princeton vs. Harvard on a neutral field.

To predict the point spread, I would usually just take the difference in the ratings for the two teams, $r_p$ and $r_h$:

$$
r_p - r_h = 15.00 - 0.75 = 14.25,
$$

which predicts that Princeton will win by about two touchdowns. But let's play with the equation a bit more:

$$
\begin{align*}
r_p - r_h &= (o_p + d_p) - (o_h + d_h) \\
&= (o_p - d_h) - (o_h - d_p) \\
&= (19.000 - (-8.375)) - (9.125 - (-4.000)) \\
&= 27.375 - 13.125 \\
&= 14.25,
\end{align*}
$$

which is the same answer we got before. However, in this case, we also get a prediction of the game score (27.375 to 13.125), not just the spread. This interpretation of the equation is nice because it gives us the predicted number of points each team will score against each other based on the strengths of their respective offenses and defenses. For example $o_p - d_h$ is the predicted number of points the Princeton offense will score on the Harvard defense. 

### Home-field Advantage 

It is quite easy to add in home-field advantage into the Massey ratings systems. We simply seek to find the value of some home advantage, which we call HA.

Let's go back to our original system of equations:

$$ 
\begin{align*}
r_p - r_c &= 35 \\
r_h - r_p &=  3 \\
r_y - r_c &= 8 \\
r_h - r_c &= 7 \\
r_p - r_y &= 28 \\
r_y - r_h &= 7 \\
\end{align*}
$$

What we do is add HA to the left side of all equations in which the winning team was at home, and we subtract HA from the left side of those equations for which the winning team was away.

The result looks like this:

$$ 
\begin{align*}
r_p - r_c - HA &= 35 \\
r_h - r_p - HA &=  3 \\
r_y - r_c - HA &= 8 \\
r_h - r_c + HA &= 7 \\
r_p - r_y - HA &= 28 \\
r_y - r_h - HA &= 7 \\
\end{align*}
$$

(Note: we are in the somewhat odd situation that the home team lost in 5 of the 6 league games).

Encoding this in matrix form, we got the following system:

$$
\begin{bmatrix}
  0 & 1 & 0 & -1 & -1 \\
  1 & -1 & 0 & 0 & -1 \\
  0 & 0 & 1 & -1 & -1 \\
  1 & 0 & 0 & -1 & 1\\
  0 & 1 & -1 & 0 & -1\\
  -1 & 0 & 1 & 0 & 1
\end{bmatrix} 
\begin{bmatrix}
  r_h \\
  r_p \\
  r_y \\
  r_c \\
  HA
\end{bmatrix}
= 
\begin{bmatrix}
  35 \\
  3 \\
  8 \\
  7 \\
  28 \\
  7
\end{bmatrix}
$$

In [ ]:
X_HA = np.array([
        [0, 1, 0, -1, -1],
        [1, -1, 0, 0, -1],
        [0, 0, 1, -1, -1],
        [1, 0, 0, -1, 1],
        [0, 1, -1, 0, -1],
        [-1, 0, 1, 0, -1]
    ])
X_HA

In [ ]:
r_HA = np.linalg.pinv(X_HA).dot(y)
r_HA

In [ ]:
col_names = ['Harvard', 'Princeton', 'Yale', 'Columbia','location']
r_HA_df = pd.DataFrame(r_HA, index=col_names,columns=[''])
print(r_HA_df.to_string())

Now, again, the location coefficient is highly negative because of the small sample size and high proportion of visiting winners. Over a long season, this location would definitely become positive.

## Advantages and Disadvantages of Massey's Method ##

At its core, Massey's method is based on margin of victory. For many (such as the BCS), this is a very bad thing since what we usually care about is who wins and not by how much. Also, running up the score on a bad team should probably not improve your rating. However, people such as Jeff Sagarin note that his point based method is a better predictor than his Elo based method, which is included in the BCS ratings.

Massey's method does have some more not so obvious advantages. For example, the square matrix $M$ is easy to construct and easy to study. Similarly, $p$ is also easy to construct. However, it is sort of annoying that we need to alter $M$ to get the method to work.

Additionally, Massey's method automatically deals with ties and also does not require each team to play the same number of games. In other methods, these details can be a pain to deal with. 

## The Connection Between Massey's Method and Linear Regression

I am now going to try to motivate Massey's Method a bit differently. At YUSAG, we use a linear regression model for our rankings. Linear regression is a classic statistical method that also relies on the method of least squares in determining the proper paramemters. Below I will attempt to show that Massey's Method is equivalent to linear regression, which offers a nice, alternative interpretation for why it works. 

(Note, what follows below is not exactly how the YUSAG linear regression model works. For that I would suggest [this notebook](https://github.com/mc-robinson/YUSAG_football_model/blob/master/YUSAG_FCS_football_linear_model.ipynb).)

### Basics of Linear Regression

Here, I am going to provide a very, very brief overview of how linear regression works. I suggest you consult any of the countless excellent resources on the topic if you wish to dive deeper.

#### The Form of Multiple Linear Regression

In linear regression, we simply attempt to predict a scalar variable $\hat y$ as the weighted sum of a bunch of input features (explanatory variables) and a bias term (the interecept). The general form is:

$$
\hat y = w_0 + w_1 x_1 + w_2 x_2 + \cdots + w_n x_n
$$
where:
* $\hat y$ is the predicted response variable
* $x_i$ is the i'th feature value
* $w_j$ is the j'th model parameter
    * $w_0$ is the bias term
    * $w_1, w_2,...,w_n$ are the feature weights
* $n$ is the number of features

So how do we figure out the values of the model coefficients $w_0,...,w_n$? The answer is that we learn these parameters when we train the linear model on the data. In fact, for linear regression, the fit is determined using the least squares criterion. That is, we seek the parameters that minimize the sum of squared errors. Once the model is trained, and the best parameters are learned, we can use the model for prediction.


#### The Data ####

For this linear regression problem, I am going to load the data from the 2016 IV season in a specialized format.

In [ ]:
lin_reg_df = pd.read_csv('IV_linear_regression_data.csv')
lin_reg_df

Some explanation of this data is in order. Each row represents a game. Within each row, the feature value for the winning team is $1$ and the value for the losing team is $-1$. All the other teams receive a value of $0$. The `location` column corresponds to the location of the winning team, with $1$ being home, $-1$ being away, and $0$ being neutral. Lastly, the `MOV` column is simply the margin of victory.

For example, in the game on 10/01/2016, Princeton beat Columbia at Columbia by 35 points.

#### Setting Up Our Model ####

Broadly, the goal of our linear regression model is to explain the margin of victory (MOV) based on the strength of the winning team and the strength of the losing team. (I am going to exclude location for now)

Let's first make our training data:

In [ ]:
X = lin_reg_df.drop(['year','month','day','location','MOV'], axis=1)
X

In [ ]:
y = lin_reg_df['MOV']
y

Now let's set up the matrix equation $Xw = y$ that we are trying to solve. Remember that we must add $x_0$ and $w_0$. I am also going to set up change the weight subscripts from $1,...,n$ to the name of the schools that the weights refer to. 

$$
\begin{bmatrix}
  1 & 0 & 1 & 0 & -1 \\
  1 & 1 & -1 & 0 & 0 \\
  1 & 0 & 0 & 1 & -1 \\
  1 & 1 & 0 & 0 & -1 \\
  1 & 0 & 1 & -1 & 0 \\
  1 & -1 & 0 & 1 & 0
\end{bmatrix} 
\begin{bmatrix}
  w_0 \\
  w_h \\
  w_p \\
  w_y \\
  w_c 
\end{bmatrix}
= 
\begin{bmatrix}
  35 \\
  3 \\
  8 \\
  7 \\
  28 \\
  7
\end{bmatrix}
$$

Mathematically, our linear model is a function of a bias term and four input features: `Harvard`, `Princeton`, `Yale`, and `Columbia`.

$$
\tt MOV = w_0 + (w_h*{Harvard}) + (w_p*{Princeton}) + (w_y*\tt{Yale}) + (w_c*\tt{Columbia})
$$

Let's examine what we get from multiplying the first row of $X$ by $w$:

$$
\tt 35 = (1*w_0) + (w_h*0) + (w_p*1) + (w_y*0) + (w_c*{-1}) \\
\tt 35 = w_0 + w_p - w_c
$$

Now, for the sake of argument, let's imagine that we set the bias term $w_0$ to be always $0$. Now the equation from the first row looks like this: 

$$
\tt 35 = w_p - w_c
$$

This looks a lot like the ideal equation from Massey's method, just with the model coefficients $w_p$ and $w_c$ replacing the ratings $r_p$ and $w_c$. That is, the difference in the model parameters for Princeton and Columbia should equal the difference in score when they play each other. If we do this for the other rows, the equations will all look similar too.

Still insisting that the bias term $w_0=0$, our full linear model looks like this: 

$$
\tt MOV = (w_h*{Harvard}) + (w_p*{Princeton}) + (w_y*\tt{Yale}) + (w_c*\tt{Columbia})
$$

And now that we no longer care about finding $w_0$, our matrix equation $Xw = y$ looks like this:

$$
\begin{bmatrix}
  0 & 1 & 0 & -1 \\
  1 & -1 & 0 & 0 \\
  0 & 0 & 1 & -1 \\
  1 & 0 & 0 & -1 \\
  0 & 1 & -1 & 0 \\
  -1 & 0 & 1 & 0
\end{bmatrix} 
\begin{bmatrix}
  w_h \\
  w_p \\
  w_y \\
  w_c
\end{bmatrix}
= 
\begin{bmatrix}
  35 \\
  3 \\
  8 \\
  7 \\
  28 \\
  7
\end{bmatrix}
$$

Look familiar? That's because this is exactly the matrix equation we got using Massey's method, just with the ratings now called weights. So this shows that Massey's method is just linear regression with the bias term set to $0$. Again, we can just solve using the method of least squares and get the desired weights, which can be interpreted as ratings.

#### Finding the Linear Regression Parameters ####

As we did before, this is an overdetermined system and can be solved using the least squares method and the normal equations. I am going to train the model using the linear regression class from scikit-learn. I will also set the `fit-intercept` parameter to `False` so that the intercept is set to be $0$.

In [ ]:
from sklearn.linear_model import LinearRegression
lin_reg = LinearRegression(fit_intercept=False)
lin_reg.fit(X, y)

In [ ]:
# print the coefficients
print(lin_reg.intercept_)
print(lin_reg.coef_)

In [ ]:
# get the R^2 value
r_squared = lin_reg.score(X, y)
print('R^2 on the training data:')
print(r_squared)

In [ ]:
# get the coefficients for each feature
coef_data = list(zip(X.columns,lin_reg.coef_))
coef_df = pd.DataFrame(coef_data,columns=['feature','feature_coef'])
coef_df.head()

As you can see, this is exactly what we got when we did the Massey ratings the old way! The coefficients of the linear regression model are precisely the ratings. And we didn't even need to do any special modifications to the matrix.

Similarly, if we want to account for home-field advantage, all you have to do is include `location` as a feauture in the model. 

In [ ]:
X_HA = lin_reg_df.drop(['year','month','day','MOV'], axis=1)
X_HA

In [ ]:
from sklearn.linear_model import LinearRegression
lin_reg_HA = LinearRegression(fit_intercept=False)
lin_reg_HA.fit(X_HA, y)

In [ ]:
# print the coefficients
print(lin_reg_HA.intercept_)
print(lin_reg_HA.coef_)

In [ ]:
# get the coefficients for each feature
coef_data = list(zip(X_HA.columns,lin_reg_HA.coef_))
coef_df = pd.DataFrame(coef_data,columns=['feature','feature_coef'])
coef_df.head()

Again, this is the same answer we got before.

One other nice feature of the linear regression point of view is that we can easily adapt the model to make adjustments commonly made for linear regression. For example, if I'm afraid my model is overfitting the data, I can use [ridge regression](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html) to add some regularization to the model.

In [ ]:
from sklearn.linear_model import Ridge
ridge_reg = Ridge(fit_intercept=False)
ridge_reg.fit(X, y)

In [ ]:
# print the coefficients
print(ridge_reg.intercept_)
print(ridge_reg.coef_)

In [ ]:
# get the R^2 value
r_squared = ridge_reg.score(X, y)
print('R^2 on the training data:')
print(r_squared)

In [ ]:
# get the coefficients for each feature
coef_data = list(zip(X.columns,ridge_reg.coef_))
coef_df = pd.DataFrame(coef_data,columns=['feature','feature_coef'])
coef_df.head()

Additionally, I can use *weighted least squares* to weight more recent games more heavily, which could be advantageous (especially if using data from past seasons). Doing this in scikit_learn is extremely easy. It's also not too difficult to implement weighting from the matrix point of view (see *Who's #1* by Langville and Meyer for details).

## References ##

Massey's Method:

* Langville, Amy N. , and Carl D. Meyer. *Who’s # 1?: The Science of Rating and Ranking* Princeton: Princeton University Press, 2012. Print.
* Chuck Wessell's helpful [lecture notes](http://public.gettysburg.edu/~cwessell/RankingPage/massey.pdf) on the method

Linear Algebra: 
* Agustinus Kristiadi's [blog post](https://wiseodd.github.io/techblog/2017/04/14/normal-equation/) on the normal equations
* Ian Goodfellow's review of linear algebra in his [Deep Learning book](http://www.deeplearningbook.org/contents/linear_algebra.html)

Linear Regression:
* *Hands-On Machine Learning with Scikit-Learn and TensorFlow* by Aurélien Géron
* *Learning From Data* by Abu-Mostafa et al. 
* *An Introduction to Statistical Learning* by James et al.
* Kevin Markham's [notebook on linear regression in scikit-learn](https://github.com/justmarkham/DAT8/blob/master/notebooks/10_linear_regression.ipynb)